In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
# DATA_PATH = Path.cwd().parent / "ML project" 
pd.set_option("display.max_columns", 100)
from sklearn.metrics import f1_score

In [2]:
train_df = pd.read_csv(
     "./train.csv", 
    index_col="tripid"
)

train_df['label'] = train_df['label'].map({"correct":1, "incorrect":0})
train_df_cols = train_df.columns.values
li = list(train_df_cols)
features_df = train_df[li[0:-1]]
labels_df = train_df[li[-1]]
# features_df

In [3]:
from math import sin, cos, sqrt, atan2, radians
def calculate_distance(**kwargs):
    R = 6373.0

    lat1 = radians(kwargs['pick_lat'])
    lon1 = radians(kwargs['pick_lon'])
    lat2 = radians(kwargs['drop_lat'])
    lon2 = radians(kwargs['drop_lon'])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c


In [4]:
def format_df(dataFrame):
    
    df = dataFrame
    df = df[~df.index.duplicated()]
    
    #calculatre the distance
    df['distance'] = [calculate_distance(**df[['pick_lat', 'pick_lon', 'drop_lat', 'drop_lon']].iloc[i].to_dict()) for i in range(df.shape[0])]
    
    #calculate the time difference between pickup and drop time in seconds
    df['time_difference'] = (pd.DatetimeIndex(df['drop_time']) - pd.DatetimeIndex(df['pickup_time'])).seconds
    
    #fill the missing values for the duration  
    df.loc[(pd.isnull(df.duration)), 'duration'] = df['duration'].mean()

    #fill the missing values for additional_fare
    df.loc[(pd.isnull(df.additional_fare)), 'additional_fare'] = df['additional_fare'].mean()
    
    #fill the missing values for meter_waiting_till_pickup
    df.loc[(pd.isnull(df.meter_waiting_till_pickup)), 'meter_waiting_till_pickup'] = df['meter_waiting_till_pickup'].mean()
    
    #fill the missing values for meter_waiting
    df.loc[(pd.isnull(df.meter_waiting)), 'meter_waiting'] = df['meter_waiting'].mean()
    

    #fill the missing values for meter_waiting_fare
    df.loc[(pd.isnull(df.meter_waiting_fare)), 'meter_waiting_fare'] = df['meter_waiting_fare'].mean()

    #fill the missing values for fare
    df.loc[(pd.isnull(df.fare)), 'fare'] = df['fare'].mean()
    

    df = df.drop(['pickup_time','drop_time','pick_lat','pick_lon','drop_lat','drop_lon'], axis = 1) 
    
    
    return df

In [5]:
newdf = format_df(features_df)
labels_df = labels_df[~labels_df.index.duplicated()]
newdf

C:\Users\X512\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\X512\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\X512\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare,distance,time_difference
tripid,,,,,,,,
189123628,10.50000,834.000000,56.000000,0.000000,64.000000,270.32,5.094369,840
189125358,10.50000,791.000000,47.000000,0.000000,134.000000,197.85,3.169052,780
189125719,10.50000,1087.000000,80.000000,0.000000,61.000000,301.64,6.307375,1080
189127273,10.50000,598.000000,271.000000,15.663800,68.000000,82.30,0.862217,600
189128020,13.72022,1702.923634,629.040422,32.057597,112.468446,358.39,8.150340,1020
...,...,...,...,...,...,...,...,...
213803193,10.50000,838.000000,93.000000,5.421900,451.000000,198.26,2.106037,840
213812756,10.50000,2151.000000,428.000000,0.000000,39.000000,581.23,10.871789,2160
213813930,10.50000,263.000000,9.000000,0.000000,110.000000,76.20,1.045847,240


In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier

from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_curve, roc_auc_score

from sklearn.preprocessing import RobustScaler

RANDOM_SEED = 6  

In [7]:
scaler = RobustScaler()
newdf[['additional_fare', 'duration','meter_waiting','meter_waiting_fare','meter_waiting_till_pickup','fare','distance','time_difference','plain_distance']] = scaler.fit_transform(newdf[['additional_fare', 'duration','meter_waiting','meter_waiting_fare','meter_waiting_till_pickup','fare','distance','time_difference','plain_distance']])
# numeric_cols = newdf.columns[newdf.dtypes != "object"].values

KeyError: "['plain_distance'] not in index"

In [ ]:
X_train, X_eval, y_train, y_eval = train_test_split(
    newdf,
    labels_df,
    test_size=0.33,
    shuffle=True,
    stratify=labels_df,
    random_state=RANDOM_SEED
)

In [ ]:
#Importing MLPClassifier
from sklearn.neural_network import MLPClassifier

#Initializing the MLPClassifier
classifier = MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=500,activation = 'relu',solver='adam',random_state=1)

In [ ]:
#Fitting the training data to the network
classifier.fit(X_train, y_train)

In [ ]:
#Predicting y for X_val
y_pred = classifier.predict(X_eval)

In [12]:
#Importing Confusion Matrix
from sklearn.metrics import confusion_matrix
#Comparing the predictions against the actual observations in y_val
cm = confusion_matrix(y_pred, y_eval)

#Printing the accuracy
# print("Accuracy of MLPClassifier : '', accuracy(cm))
f1 = f1_score(y_eval, y_pred)
print(f1)

0.9701970682458013


In [13]:
test_features = pd.read_csv("./test.csv", 
                               index_col="tripid")

In [14]:
testdf = format_df(test_features)


In [16]:
scaler = RobustScaler()
testdf[['additional_fare', 'duration','meter_waiting','meter_waiting_fare','meter_waiting_till_pickup','fare','distance','time_difference','plain_distance']] = scaler.fit_transform(testdf[['additional_fare', 'duration','meter_waiting','meter_waiting_fare','meter_waiting_till_pickup','fare','distance','time_difference','plain_distance']])
# numeric_cols = newdf.columns[newdf.dtypes != "object"].values

In [17]:
test_probas = classifier.predict(testdf)
test_preds = pd.DataFrame(
    {'prediction':test_probas},
    index = test_features.index
)
print(test_preds)

           prediction
tripid               
213284604           1
213286352           1
213293973           1
213294622           1
213298687           1
...               ...
222856243           1
222857785           1
222858416           1
222858691           1
222860703           1

[8576 rows x 1 columns]


In [18]:
test_preds.to_csv('./mlpnew1.csv', index=True)